In [1]:
#| include: false

import os
import pandas as pd
import numpy as np
import sqlite3
# import plotly.express as px
import toml
# import psrc_theme
from pathlib import Path
import summary_data

# to show plotly figures in quarto HTML file
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"
# pio.templates.default = "simple_white+psrc_color" # set plotly template

config = summary_data.CONFIG
all_runs = summary_data.ALL_RUNS

In [2]:
# Join parcel lookup data to parcel files to do some county and district summaries
# Load parcel geography lookup from soundcast db
conn = sqlite3.connect(r'../../../../inputs/db/soundcast_inputs_2023.db')
df_geog_lookup = pd.read_sql_query("SELECT ParcelID, CountyName, district_name FROM parcel_2023_geography", conn)

# Land Use Summary
df_parcels = summary_data.load_landuse('landuse/parcels_urbansim.txt')
# Merge lookup data to parcels
df_parcels = df_parcels.merge(df_geog_lookup, left_on='parcelid', right_on='ParcelID', how='left')

## Synthetic Population

- total number of households

In [3]:
def count_by_county(output_path: str, summary_var: str, expfac_name: str, axis_name: str):
    df = summary_data.load_agg_data(output_path)

    tab = df.groupby(['source',summary_var])[expfac_name].sum().\
        unstack(summary_var).\
        rename_axis(columns={summary_var: axis_name})
    tab['Total'] = tab.sum(axis=1)

    display(tab.style.format('{:,.0f}'))

In [4]:
count_by_county('agg/dash/hh_geog.csv','hh_county','hhexpfac','Households by county')

Households by county,King,Kitsap,Outside Region,Pierce,Snohomish,Total
source,,,,,,
current run,"957,460","108,901",3,"350,141","319,624","1,736,129"


- total population

In [5]:
count_by_county('agg/dash/pptyp_county.csv','person_county','psexpfac','Population by county')

Population by county,King,Kitsap,Outside Region,Pierce,Snohomish,Total
source,,,,,,
current run,"2,303,851","275,120",8,"925,555","851,485","4,356,019"


## Student Enrollment

In [6]:
# Students by grade level
df_pivot = pd.pivot_table(df_parcels, index='source', 
                          values=['stugrd_p', 'stuhgh_p', 'stuuni_p'], 
                           aggfunc='sum')
df_pivot.rename(columns={'stugrd_p': 'Grade School', 'stuhgh_p':'High School', 'stuuni_p':'College'}, inplace=True)
df_pivot.style.format('{:,.0f}')

,Grade School,High School,College
source,,,
current run,"442,413","211,261","136,681"


## Employment

In [7]:
df_pivot = pd.pivot_table(df_parcels, index='source', 
                          values=['emptot_p','empedu_p', 'empfoo_p', 'empgov_p', 'empind_p', 'empmed_p',
                                'empofc_p', 'empoth_p', 'empret_p', 'emprsc_p', 'empsvc_p'], 
                           aggfunc='sum')
df_pivot.rename(columns={'emptot_p': 'Total', 'empedu_p': 'Education', 'empfoo_p':'Food', 'empgov_p':'Government',
                         'empind_p':'Industrial', 'empmed_p':'Medical', 'empofc_p':'Office',
                         'empoth_p':'Other', 'empret_p':'Retail', 'emprsc_p':'Resource', 'empsvc_p':'Service'}, inplace=True)
df_pivot.style.format('{:,.0f}')

,Education,Food,Government,Industrial,Medical,Office,Other,Retail,Resource,Service,Total
source,,,,,,,,,,,
current run,"167,143","157,285","222,537","341,587","242,463","582,273","155,306","176,184",0,"146,168","2,190,946"


In [8]:
#| echo: true
df_pivot = pd.pivot_table(df_parcels, index='CountyName', columns='source', values='emptot_p', aggfunc='sum')
df_pivot.style.format('{:,.0f}')

source,current run
CountyName,
King,"1,442,506"
Kitsap,"92,707"
Outside Region,0
Pierce,"334,336"
Snohomish,"321,397"


In [9]:
#| echo: true
count_by_county('agg/dash/person_worker_type.csv','person_work_county','psexpfac','Employment by county')

Employment by county,King,Kitsap,Pierce,Snohomish,Total
source,,,,,
current run,"1,433,410","101,517","353,639","338,441","2,227,007"


In [10]:
# Group total employment by district and source
df_pivot = pd.pivot_table(df_parcels, index='district_name', columns='source', values='emptot_p', aggfunc='sum')
df_pivot.style.format('{:,.0f}')

source,current run
district_name,
East Side,"420,109"
Everett-Lynwood-Edmonds,"158,328"
Kitsap,"92,707"
North Seattle-Shoreline,"147,826"
Renton-FedWay-Kent,"343,930"
S.Kitsap,"19,990"
Seattle CBD,"394,252"
South Pierce,"167,286"
Suburban Snohomish,"163,069"
